In [ ]:
import os
import glob

import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

%matplotlib inline

In [ ]:
iabp_dir = '/Data/sim/data/IABP_live'
iabp_files = sorted(glob.glob(f'{iabp_dir}/*[0-9].dat'))
print(f'Found {len(iabp_files)} IABP files')

In [ ]:
min_df_shape = 10

buoys = []
for i, ifile in tqdm(enumerate(iabp_files), total=len(iabp_files)):
    if os.path.basename(ifile).startswith('FR_'):
        df = pd.read_csv(ifile, sep=';', header=None,  usecols=[1, 4, 6, 7], names=['Year', 'DOY', 'Lat', 'Lon'])
    else:
        df = pd.read_csv(ifile, sep='\\s+', usecols=['Year', 'DOY', 'Lat', 'Lon']).dropna()
    valid_lon_lat = (df.Lon >= -180) & (df.Lon <= 180) & (df.Lat > 50) & (df.Lat <= 90)
    valid_year = (df.Year <= 2025) & (df.Year >= 1990)
    df = df[valid_lon_lat & valid_year]
    if df.shape[0] < min_df_shape:
        continue
    max_lon_dif = df.Lon.diff().abs().quantile(0.95)
    max_lat_dif = df.Lat.diff().abs().quantile(0.95)
    lon_lat_mask = (df.Lon.diff().abs() < max_lon_dif) & (df.Lat.diff().abs() < max_lat_dif)
    df = df[lon_lat_mask]
    if df.shape[0] < min_df_shape:
        continue
    df.index = pd.to_timedelta(df['DOY'], unit='D') + pd.to_datetime(df['Year'], format='%Y')
    df_1d = df.resample('1D').mean().dropna()
    if df_1d.shape[0] < min_df_shape:
        continue
    buoy = pd.DataFrame({
            'BuoyId': i,
            'Lat': df_1d['Lat'],
            'Lon': df_1d['Lon'],
            'Age': (df_1d.index - df_1d.index.min()).days,
        },
        index=df_1d.index)
    if np.isnan(buoy.Lat.values.min()):
        raise
    buoys.append(buoy)
buoys = pd.concat(buoys, axis=0)


In [ ]:
buoys.to_feather('data/buoys.feather')

In [ ]:
# Resample IABP trajectories to 1 day
buoys_resampled = buoys.resample('1D').mean().dropna()

# Save the resampled data to a new feather file
buoys_resampled.to_feather('data/buoys_resampled.feather')

# Print the first few rows of the resampled data
print(buoys_resampled.head())